In [1]:
"""Imports"""

import os
from ansys.edb.database import Database
from ansys.edb.layer import StackupLayer, LayerType
from ansys.edb.layout import Cell, CellType
from ansys.edb.net import Net
from ansys.edb.primitive import Circle, Text, Rectangle, RectangleRepresentationType

In [2]:
"""Configs"""

# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = None
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "example_Define_and_assign_bondwire_profiles.aedb")

In [ ]:
"""Launch a session. Make sure old one has been disconnected prior to starting a new one, if any"""

session = launch_session(RPC_SERVER, 50051)

In [ ]:
"""Basic setup of empty EDB."""

db = Database.create(EDB_FILE)
cell = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign1")
layout = cell.layout

"""Nets for the different cells and create the primitives"""

net = Net.create(layout, 'GND')

In [ ]:
"""Setup Layers for cell"""

# Layers in layout's layer collection
layer_gnd = StackupLayer.create('Ground', layer_type=LayerType.SIGNAL_LAYER, thickness=0.0001, elevation=0, material="copper")
layer_1 = StackupLayer.create('NewLayer', LayerType.SIGNAL_LAYER, thickness=0.0001, elevation=0.0001, material="copper")
lc = layout.layer_collection
lc.add_layers([layer_gnd, layer_1])

# Retrieve layers collection and layers
layout.layer_collection = lc
layer_gnd = lc.find_by_name('Ground')
layer_1 = lc.find_by_name('NewLayer')

In [ ]:
"""Create Primitive objects in ground layer"""

um = lambda val : val * 1e-6

text = Text.create(layout=layout,layer=layer_gnd, center_x=um(0.0), center_y=um(0.0), text="my Text")

rect = Rectangle.create(layout=layout,
                        layer=layer_gnd,
                        net=net,
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(10),
                        param2=um(10),
                        param3=um(20),
                        param4=um(20),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout,
                          layer=layer_gnd,
                          net=net,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(0.0),
                          param2=um(0.0),
                          param3=um(30),
                          param4=um(30),
                          corner_rad=0.0,
                          rotation=0.0
                          )

circle = Circle.create(layout=layout,layer=layer_gnd,net=net, center_x=um(0.0), center_y=um(0.0), radius=um(34.0))

# Add primitives to a list
primitives_created = [text, rect, rect_2, circle]

In [ ]:
"""Move Primitive objects to NewLayer"""

for primitive in primitives_created:
    primitive.layer = layer_1

In [ ]:
"""Save database if you wish."""

db.save()

In [ ]:
"""Don't forget to disconnect it!"""

session.disconnect()